# Using Poisson equations using PINNs

## Environment Setup

This notebook requires **MindSpore version >= 2.0.0** to support new APIs including: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*. Please check [MindSpore Installation](https://www.mindspore.cn/install/en) for details.

In addition, **MindFlow version >=0.1.0** is also required. If it has not been installed in your environment, please select the right version and hardware, then install it as follows.

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## Problem description

This example demonstrates how to use PINNs to solve 1D, 2D and 3D Poisson equations in different geometries. The 1D equation is defined by

$$
\Delta u = -\sin(4\pi x),
$$

The 2D equation is defined by

$$
\Delta u = -\sin(4\pi x)\sin(4\pi y),
$$

and the 3D equation is given by

$$
\Delta u = -\sin(4\pi x)\sin(4\pi y)\sin(4\pi z),
$$

The following functions satisfy the 1D, 2D and 3D equations respectively,

$$
u = \frac{1}{16\pi^2} \sin(4\pi x), \\
u = \frac{1}{32\pi^2} \sin(4\pi x)\sin(4\pi y), \\
u = \frac{1}{48\pi^2} \sin(4\pi x)\sin(4\pi y)\sin(4\pi z).
$$

If we set the Dirichlet boundary condition using these functions, they become the solution to the problems. In this example, we solve the 2D equation in rectangle, disk, triangle, L-type and pentagon. In the 3D case, we solve the problem in a tetrahedron, cylinder and cone.

## Method

MindFlow solves the problem as follows:

1. Creating the dataset.
2. Creating the neural network.
3. Setting the constraint.
4. Creating the optimizer.
5. Model training.
6. Model evaluation.

In [1]:
import time

from mindspore import context, nn, ops, jit
from mindflow import load_yaml_config

from src.model import create_model
from src.lr_scheduler import OneCycleLR
from src.dataset import create_dataset


context.set_context(mode=context.GRAPH_MODE, save_graphs=False, device_target="GPU")
use_ascend = context.get_context(attr_key="device_target") == "Ascend"

# Load config
file_cfg = "configs/poisson_cfg.yaml"
config = load_yaml_config(file_cfg)

## Creating the dataset

This example creates the dataset by ramdom sampling in the domain and on the boundaries (See ``src/dataset.py`` for the implementation). Set ``geom_name`` to select geometry, which can be 'triangle', 'pentagon', 'tetrahedron', 'cylinder', and 'cone'.

In [2]:
geom_name = "interval"
ds_train, n_dim = create_dataset(geom_name, config)

## Creating the neural network

This example adopts a MLP with 3 hidden layers with the following features:

- Using $f(x) = x \exp(-x^2/(2e)) $ as the activation function.
- Using weight normalization for the last layer.
- Employing the ``HeUniform`` from ``mindspore`` for the initialization of all weights.

See ``src/model.py`` for the details.

In [3]:
model = create_model(**config['model'][f'{n_dim}d'])

## Setting the constraint

When applying ``mindflow`` to solving PDEs, we need to write a subclass of ``mindflow.PDEWithLloss`` to define the governing equation, boundary condition and loss function. In this example, we adopt the L2 loss in the domain and on the boundaries. Both losses are combined using the multitarget loss defined by ``MTLWeightedLoss``.

In [ ]:
"""Define the boundary condition."""
import logging
import sympy
def dirichlet(n_dim, in_vars, out_vars):
    '''
        1d dirichlet boundary: a * u(x) = b
        2d dirichlet boundary: a * u(x, y) = b
        3d dirichlet boundary: a * u(x,y,z) = b
        note: a is a constant, the variables x, y, and z
              are in the boundary region, and b can be a function or a constant.
    '''
    bc_term = 1
    for var in in_vars:
        bc_term *= sympy.sin(4*sympy.pi*var)
    try:
        bc_term *= 1/(16*n_dim*sympy.pi*sympy.pi)
    except ZeroDivisionError:
        logging.error("Error: The divisor cannot be zero!")

    bc_eq = out_vars[0] - bc_term  # u(x) - bc_term
    equations = {"bc": bc_eq}
    return equations


def robin(n_dim, in_vars, out_vars):
    '''
        1d robin boundary: a * u(x) + b * u'(x) = c
        2d robin boundary: a * u(x, y) + b * u'(x, y) = c
        3d robin boundary: a * u(x,y,z) + b * u'(x,y,z) = c
        note: a, b is a constant, the variables x, y, and z
              are in the boundary region,
              u' is the number of external wizards of the function,
              and c can be a function or a constant.
    '''
    bc_term = 1
    u_x = 0
    bc_term_u = 0
    for var in in_vars:
        bc_term_ux = 1
        u_x += sympy.diff(out_vars[0], var)  # Derivation
        bc_term *= sympy.sin(4*sympy.pi*var)
        for i in in_vars:
            if i != var:  # Partial conduction
                bc_term_ux *= sympy.sin(4*sympy.pi*i)
            elif i == var:
                bc_term_ux *= sympy.cos(4*sympy.pi*i)
        bc_term_u += bc_term_ux
    try:
        bc_term *= 1/(16*n_dim*sympy.pi*sympy.pi)  # function
        bc_term_u *= 1/(4*n_dim*sympy.pi)
    except ZeroDivisionError:
        logging.error("Error: The divisor cannot be zero!")

    # u(x) + u'(x) - bc_term - bc_term_u
    bc_eq = out_vars[0] + u_x - bc_term - bc_term_u
    equations = {"bc": bc_eq}
    return equations


def periodic(n_dim, in_vars, out_vars):
    '''
        Periodic boundary conditions are a special case of Robin boundary conditions.
        1d periodic boundary: a * u(x) + b * u'(x) = a * u(x+T) + b * u'(x+T) = c
        2d periodic boundary: a * u(x,y) + b * u'(x,y) = a * u(x+T1,y+T2) + b * u'(x+T1,y+T2) = c
        3d periodic boundary: a * u(x,y,z) + b * u'(x,y,z) = a * u(x+T1,y+T2,z+T3) + b * u'(x+T1,y+T2,z+T3) = c
        note: a, b is a constant, the variables x, y, and z
              are in the boundary region,
              T1, T2, T3 corresponds to the period size of each variable in the defined interval,
              u' is the number of external wizards of the function,
              and c can be a function or a constant.
    '''
    _ = out_vars
    bc_term = 1
    for _ in in_vars:
        bc_term *= 2
    try:
        bc_term *= 1/(16*n_dim*sympy.pi*sympy.pi)
    except ZeroDivisionError:
        logging.error("Error: The divisor cannot be zero!")
    bc_eq = bc_term  # bc_term
    equations = {"bc": bc_eq}
    return equations


bc_type = {
    "dirichlet": dirichlet,
    "robin": robin,
    "periodic": periodic,
}


def get_bc(bc):
    '''return boundary condition'''
    try:
        boundary = bc_type[bc]
    except KeyError:
        logging.error("Wrong boundary name.")
    return boundary


In [4]:
import sympy
from mindspore import numpy as ms_np
from mindflow import PDEWithLoss, MTLWeightedLoss, sympy_to_mindspore


def pde(out_vars, in_vars):
    poisson = 0
    src_term = 1
    sym_u = out_vars[0]
    for var in in_vars:
        poisson += sympy.diff(sym_u, (var, 2))
        src_term *= sympy.sin(4 * sympy.pi * var)
    poisson += src_term
    equations = {"poisson": poisson}
    return equations

class Poisson(PDEWithLoss):
    """Define the loss of the Poisson equation."""

    def __init__(self, model, n_dim, pde, bc=None):
        if n_dim == 1:
            var_str = "x,"
        elif n_dim == 2:
            var_str = "x y"
        elif n_dim == 3:
            var_str = "x y z"
        else:
            raise ValueError("`n_dim` can only be 2 or 3.")
        self.in_vars = sympy.symbols(var_str)
        self.out_vars = (sympy.Function("u")(*self.in_vars),)
        self.bc = bc
        self.pde_fun = pde
        super(Poisson, self).__init__(model, self.in_vars, self.out_vars)
        if self.bc:  # boundary
            self.bc_nodes = sympy_to_mindspore(
                self.bc(n_dim, self.in_vars, self.out_vars), self.in_vars, self.out_vars)
        self.loss_fn = MTLWeightedLoss(num_losses=2)

    def pde(self):
        """Define the gonvering equation."""
        return self.pde_fun(self.out_vars, self.in_vars)  # equations

    def get_loss(self, pde_data, bc_data):
        """Define the loss function."""
        res_pde = self.parse_node(self.pde_nodes, inputs=pde_data)
        res_bc = self.parse_node(self.bc_nodes, inputs=bc_data)
        loss_pde = ms_np.mean(ms_np.square(res_pde[0]))
        loss_bc = ms_np.mean(ms_np.square(res_bc[0]))
        return self.loss_fn((loss_pde, loss_bc))


# Create the problem
problem = Poisson(model, n_dim, pde, get_bc(config['data']['BC']['BC_type']))

poisson: sin(4*pi*x)*sin(4*pi*y) + Derivative(u(x, y), (x, 2)) + Derivative(u(x, y), (y, 2))
    Item numbers of current derivative formula nodes: 3
bc: u(x, y) - sin(4*pi*x)*sin(4*pi*y)/(32*pi**2)
    Item numbers of current derivative formula nodes: 2


## Creating the optimizer

This example applies the Adam optimizer, and adopts the dynamic learning rate proposed by [Super-Convergence: Very Fast Training of Neural Networks Using Large Learning Rates](https://arxiv.org/abs/1708.07120). See ``src/lr_scheduler.py`` for the implementation of the dynamic learning rate.

In [5]:
n_epochs = 50

params = model.trainable_params() + problem.loss_fn.trainable_params()
steps_per_epoch = config['data']['domain']['size']//config["data"]["train"]["batch_size"]
learning_rate = OneCycleLR(total_steps=steps_per_epoch*n_epochs, **config['optimizer'])
optimizer = nn.Adam(params, learning_rate=learning_rate)

## Model training

With MindSpore version >= 2.0.0, we can use the functional programming for training neural networks.

In [6]:
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

def train():

    if use_ascend:
        from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
        loss_scaler = DynamicLossScaler(1024, 2, 100)
        auto_mixed_precision(model, 'O1')
    else:
        loss_scaler = None

    # the loss function receives 2 data sources: pde and bc
    def forward_fn(pde_data, bc_data):
        loss = problem.get_loss(pde_data, bc_data)
        if use_ascend:
            loss = loss_scaler.scale(loss)

        return loss
    # Create
    grad_fn = ops.value_and_grad(forward_fn, None, opt.parameters, has_aux=False)

    @jit
    def train_step(pde_data, bc_data):
        loss, grads = grad_fn(pde_data, bc_data)
        if use_ascend:
            loss = loss_scaler.unscale(loss)
            is_finite = all_finite(grads)
            if is_finite:
                grads = loss_scaler.unscale(grads)
                loss = ops.depend(loss, opt(grads))
            loss_scaler.adjust(is_finite)
        else:
            loss = ops.depend(loss, opt(grads))
        return loss

    def train_epoch(model, dataset, i_epoch):
        n_step = dataset.get_dataset_size()
        model.set_train()
        for i_step, (pde_data, bc_data) in enumerate(dataset):
            local_time_beg = time.time()
            loss = train_step(pde_data, bc_data)

            if i_step%50 == 0 or i_step + 1 == n_step:
                print("\repoch: {}, loss: {:>f}, time elapsed: {:.1f}ms [{}/{}]".format(
                    i_epoch, float(loss), (time.time() - local_time_beg)*1000, i_step + 1, n_step))

    for i_epoch in range(n_epochs):
        train_epoch(model, ds_train, i_epoch)

In [7]:
time_beg = time.time()
train()
print("End-to-End total time: {} s".format(time.time() - time_beg))

epoch: 0, loss: 1.527029, time elapsed: 12050.1ms [1/200]
epoch: 0, loss: 1.468655, time elapsed: 52.4ms [51/200]
epoch: 0, loss: 1.442717, time elapsed: 52.3ms [101/200]
epoch: 0, loss: 1.430150, time elapsed: 52.4ms [151/200]
epoch: 0, loss: 1.420228, time elapsed: 53.4ms [200/200]
epoch: 1, loss: 1.419910, time elapsed: 53.0ms [1/200]
epoch: 1, loss: 1.407040, time elapsed: 52.5ms [51/200]
epoch: 1, loss: 1.386505, time elapsed: 52.4ms [101/200]
epoch: 1, loss: 1.362307, time elapsed: 52.4ms [151/200]
epoch: 1, loss: 1.349054, time elapsed: 52.5ms [200/200]
epoch: 2, loss: 1.349143, time elapsed: 53.7ms [1/200]
epoch: 2, loss: 1.336657, time elapsed: 52.7ms [51/200]
epoch: 2, loss: 1.323158, time elapsed: 52.6ms [101/200]
epoch: 2, loss: 1.307419, time elapsed: 52.9ms [151/200]
epoch: 2, loss: 1.289993, time elapsed: 52.7ms [200/200]
epoch: 3, loss: 1.289594, time elapsed: 53.5ms [1/200]
epoch: 3, loss: 1.270476, time elapsed: 52.4ms [51/200]
epoch: 3, loss: 1.246817, time elapsed: 

## Model evaluation

We can use the following function to calculate the relative L2 error.

In [8]:
from src.utils import calculate_l2_error

n_samps = 5000 # Number of test samples
ds_test, _ = create_dataset(geom_name, config, n_samps)
calculate_l2_error(model, ds_test, n_dim)

Relative L2 error (domain): 0.0310
Relative L2 error (bc): 0.0833



## Result

In the 2D rectangular sampling area, the model training result is as follows:
<p align = "center">
<img src="./images/dirichlet-rectangle.png" width="250"/>
</p>